In [ ]:
# %%
# Import necessary libraries for data handling, model utilization, and visualization
import numpy as np
import pandas as pd
import yfinance as yf  # For collecting financial data
import matplotlib.pyplot as plt
from typing import List, Dict, Optional, Tuple
from datetime import datetime, timedelta
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from scipy.optimize import minimize
from collections import deque
# Import the custom Model class
from Model import Model

# Set the random seed for reproducibility across numpy and tensorflow
np.random.seed(123)
tf.random.set_seed(123)

plt.style.use('seaborn-darkgrid')

# Define the tickers and date range with consideration of trading days
TICKERS = ['AGG','DBC','VTI','^VIX']

# Approximate number of trading days per year (useful for annualizing returns)
TRADING_DAYS_PER_YEAR = 252

# Define transaction cost rate
C = 0.0001  # 0.01%

# Confirm setup
print("Setup complete: libraries imported, random seed set, and tickers defined.")

Setup complete: libraries imported, random seed set, and tickers defined.


C:\Users\wiztu\AppData\Local\Temp\ipykernel_18988\1252333898.py:23: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')


In [13]:
# %%
# Data Collection Step
# Objective: Fetch historical adjusted close prices for defined tickers and date range

# Download data using yfinance for the specified tickers and date range
def get_data(tickers, start_date, end_date):
    """
    Retrieves historical adjusted close prices for the given tickers and date range.
    
    Parameters:
    - tickers: List of stock ticker symbols
    - start_date: Start date for historical data
    - end_date: End date for historical data
    
    Returns:
    - DataFrame of adjusted close prices, with each column representing a ticker
    """
    # Fetch data from yfinance
    data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
    
    # Drop rows with missing values, if any, to ensure data continuity
    data.dropna(inplace=True)
    
    return data

# Fetch the data and display a quick preview
data = get_data(TICKERS, '2006-01-01', '2020-04-30')
print("Data fetched successfully. Sample data:")
print(data.head())

# Confirm data spans the expected range and has the expected number of columns
print(f"Data covers {len(data)} trading days with {len(data.columns)} assets.")

[*********************100%%**********************]  4 of 4 completed

Data fetched successfully. Sample data:
Ticker            AGG        DBC        VTI   ^VIX
Date                                              
2006-02-06  56.300137  20.889500  44.654274  13.04
2006-02-07  56.260841  20.285254  44.219498  13.59
2006-02-08  56.232685  20.198933  44.537624  12.83
2006-02-09  56.266430  20.388840  44.452778  13.12
2006-02-10  56.148418  20.017662  44.544716  12.87
Data covers 3582 trading days with 4 assets.


In [14]:
# %%
def preprocess_data(data, rolling_window=50):
    """
    Prepares data by calculating 50-day rolling averages and returns.

    Parameters:
    - data: DataFrame of historical adjusted close prices for assets
    - rolling_window: Window size for the rolling average

    Returns:
    - normalized_data: Smoothed prices, normalized to start at 1 for each asset
    - returns: Smoothed returns using a rolling mean of percentage changes
    """
    # Calculate rolling mean for prices and returns to smooth the data
    smoothed_prices = (data.rolling(window=rolling_window).mean()).dropna()
    smoothed_returns = (data.pct_change().rolling(window=rolling_window).mean()).dropna()
    # Normalize prices to start each asset's time series at 1
    normalized_data = smoothed_prices / smoothed_prices.iloc[0]
    
    return normalized_data, smoothed_returns


# Run preprocessing and display sample data
normalized_data, smoothed_returns = preprocess_data(data)
print("Data preprocessing complete. Sample normalized data:")
print(normalized_data.head())
print("\nSample daily returns:")
print(smoothed_returns.head())


Data preprocessing complete. Sample normalized data:
Ticker           AGG       DBC       VTI      ^VIX
Date                                              
2006-04-18  1.000000  1.000000  1.000000  1.000000
2006-04-19  0.999786  1.001443  1.000798  0.997130
2006-04-20  0.999555  1.003226  1.001815  0.993875
2006-04-21  0.999332  1.005552  1.002683  0.991806
2006-04-24  0.999172  1.007249  1.003528  0.989520

Sample daily returns:
Ticker           AGG       DBC       VTI      ^VIX
Date                                              
2006-04-19 -0.000212  0.001441  0.000813 -0.001952
2006-04-20 -0.000229  0.001788  0.001035 -0.002230
2006-04-21 -0.000221  0.002295  0.000882 -0.001198
2006-04-24 -0.000157  0.001709  0.000862 -0.001374
2006-04-25 -0.000182  0.002136  0.000757 -0.000993


In [15]:
# %%
class Portfolio:
    def __init__(self, initial_cash: float, assets: pd.DataFrame):
        """
        Initializes the Portfolio object.

        Parameters:
        - initial_cash: The starting value of the portfolio in cash.
        - assets: DataFrame of asset prices (historical data).
        """
        self.initial_cash = initial_cash
        self.current_value = initial_cash
        self.assets = assets  # Historical price data for the assets
        self.weights = np.zeros(len(assets.columns))  # Initialize weights as zero
        self.portfolio_history = []  # To track portfolio value over time
        self.rebalancing_dates = []  # To store rebalancing dates

    def rebalance(self, new_weights: np.array, target_volatility = None):
        """
        Rebalances the portfolio according to new weights.

        Parameters:
        - new_weights: Numpy array of asset allocations.
        """
        if len(new_weights) != len(self.assets.columns):
            raise ValueError("Number of weights must match the number of assets.")
        self.weights = new_weights
        if target_volatility is not None:
            self.apply_volatility_scaling(target_volatility=target_volatility,rolling_window = 50)

    def calculate_initial_shares(self, initial_cash, initial_prices):
        """
        Calculates the number of shares for each asset at the start of the testing period based on
        initial cash and allocation weights.

        Parameters:
        - initial_cash: The starting cash value of the portfolio.

        Returns:
        - shares: Dictionary with tickers as keys and the initial number of shares as values.
        """
        # Calculate the dollar amount allocated to each asset
        dollar_allocation = initial_cash * self.weights

        # Calculate the number of shares for each asset
        shares = (dollar_allocation // initial_prices).astype(int)  # Floor division to get whole shares

        # Return as a dictionary for easy readability
        return dict(zip(self.assets.columns, shares))
    
    def calculate_daily_returns(self):
        """
        Applies the current weights to asset returns and updates portfolio value over time.
        """
        # Calculate daily returns for each asset
        daily_returns = self.assets.pct_change().dropna()
        
        # Calculate portfolio returns by applying weights
        portfolio_returns = np.dot(daily_returns, self.weights)

        # Track the portfolio's value over time by compounding the returns
        for daily_ret in portfolio_returns:
            self.current_value *= (1 + daily_ret)
            self.portfolio_history.append(self.current_value)

    def update_portfolio_value(self, date):
        """
        Updates the portfolio value for a single date.
        """
        # Get the index of the date
        date_index = self.assets.index.get_loc(date)
        if date_index == 0:
            # First day, no previous day to compute return
            self.portfolio_history.append(self.current_value)
            return
        # Get the asset returns for that day
        previous_date = self.assets.index[date_index - 1]
        daily_return = self.assets.loc[date] / self.assets.loc[previous_date] - 1
        # Calculate portfolio return
        portfolio_return = np.dot(daily_return.values, self.weights)
        # Update portfolio value
        self.current_value *= (1 + portfolio_return)
        # Append to history
        self.portfolio_history.append(self.current_value)


    def track_portfolio_performance(self):
        """
        Tracks and prints the portfolio performance over time.
        """
        for date, value in zip(self.assets.index[1:], self.portfolio_history):
            print(f"Date: {date}, Portfolio Value: {value}")
    
    def reset(self):
        self.current_cash = self.initial_cash
        self.assets = self.initial_assets.copy()
        self.weights = np.zeros(len(self.assets.columns))  # Reset to no investments
        self.portfolio_history = []
        self.rebalancing_dates = []
        return self.assets.iloc[0].values

    def get_portfolio_value(self):
        """
        Returns the current value of the portfolio.
        """
        return self.current_value
    
    def plot_portfolio_value(self):
        """
        Plots the portfolio value over time.
        """
        plt.figure(figsize=(10, 6))
        plt.plot(self.assets.index[1:], self.portfolio_history, label="Portfolio Value")
        plt.title("Portfolio Value Over Time")
        plt.xlabel("Date")
        plt.ylabel("Portfolio Value")
        plt.legend()
        plt.show()


In [16]:
# %%
def calculate_metrics(portfolio_values):
    """
    Calculates performance metrics for the portfolio.

    Parameters:
    - portfolio_values: List of daily portfolio values over the testing period.

    Returns:
    - metrics: Dictionary containing various performance metrics.
    """
    # Convert portfolio values to daily returns
    portfolio_returns = np.diff(portfolio_values) / portfolio_values[:-1]
    
    # Number of days
    N = len(portfolio_returns)

    # Calculate Sharpe Ratio
    mean_return = np.mean(portfolio_returns)
    std_dev = np.std(portfolio_returns)
    sharpe_ratio = mean_return / std_dev * np.sqrt(TRADING_DAYS_PER_YEAR)
    
    # Calculate Sortino Ratio
    downside_returns = portfolio_returns[portfolio_returns < 0]
    downside_std_dev = np.std(downside_returns) if len(downside_returns) > 0 else 0
    sortino_ratio = mean_return / downside_std_dev * np.sqrt(TRADING_DAYS_PER_YEAR) if downside_std_dev != 0 else np.nan
    
    # Calculate Maximum Drawdown
    cumulative_max = np.maximum.accumulate(portfolio_values)
    drawdowns = (cumulative_max - portfolio_values) / cumulative_max
    max_drawdown = np.max(drawdowns)
    
    # Expected return (annualized)
    cumulative_return = (portfolio_values[-1] / portfolio_values[0]) - 1
    annualized_return = (1 + cumulative_return) ** (TRADING_DAYS_PER_YEAR / N) - 1

    # Standard deviation of returns (annualized)
    annualized_std = std_dev * np.sqrt(TRADING_DAYS_PER_YEAR)

    # Percentage of positive returns
    positive_returns = portfolio_returns[portfolio_returns > 0]
    percentage_positive = len(positive_returns) / len(portfolio_returns) * 100

    # Average profit / average loss (profit/loss ratio)
    average_profit = np.mean(portfolio_returns[portfolio_returns > 0]) if len(positive_returns) > 0 else 0
    average_loss = np.mean(portfolio_returns[portfolio_returns < 0]) if len(portfolio_returns[portfolio_returns < 0]) > 0 else 0
    profit_loss_ratio = (average_profit / -average_loss) if average_loss != 0 else np.nan

    metrics = {
        "Annualized Return": annualized_return,
        "Annualized Std Dev": annualized_std,
        "Sharpe Ratio": sharpe_ratio,
        "Sortino Ratio": sortino_ratio,
        "Max Drawdown": max_drawdown,
        "% Positive Returns": percentage_positive,
        "Profit/Loss Ratio": profit_loss_ratio
    }
    
    return metrics


In [ ]:
# %%
def equal_weighted_strategy(returns):
    """
    Creates an equal-weighted portfolio.

    Parameters:
    - returns: DataFrame of daily returns for each asset.

    Returns:
    - equal_weights: Numpy array of equal weights for each asset.
    """
    num_assets = returns.shape[1]
    equal_weights = np.ones(num_assets) / num_assets
    return equal_weights
# Define function to get MV weights
def get_mv_weights(mu, sigma, risk_free_rate=0):
    """
    Computes the weights for mean-variance optimization (MV) that maximize the Sharpe ratio.

    Parameters:
    - mu: Expected returns vector
    - sigma: Covariance matrix of returns
    - risk_free_rate: Risk-free rate (default 0)

    Returns:
    - weights: Optimized portfolio weights
    """
    num_assets = len(mu)

    # Define the objective function: negative Sharpe ratio
    def neg_sharpe_ratio(weights):
        port_return = np.dot(weights, mu)
        port_vol = np.sqrt(np.dot(weights.T, np.dot(sigma, weights)))
        sharpe_ratio = (port_return - risk_free_rate) / (port_vol + 1e-8)
        return -sharpe_ratio  # Negative because we minimize

    # Constraints: sum of weights = 1
    constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}

    # Bounds: weights between 0 and 1 (long-only portfolio)
    bounds = tuple((0, 1) for _ in range(num_assets))

    # Initial guess: equal weights
    initial_weights = np.ones(num_assets) / num_assets

    # Optimize
    result = minimize(neg_sharpe_ratio, initial_weights, bounds=bounds, constraints=constraints)
    
    # Ensure the solution is valid
    if not result.success:
        return initial_weights
    return result.x

# Define function to get MD weights
def get_md_weights(sigma):
    """
    Computes the weights for maximum diversification (MD) that maximize the diversification ratio.

    Parameters:
    - sigma: Covariance matrix of returns

    Returns:
    - weights: Optimized portfolio weights
    """
    num_assets = sigma.shape[0]
    asset_vols = np.sqrt(np.diag(sigma))

    # Define the objective function: negative diversification ratio
    def neg_diversification_ratio(weights):
        weights = np.array(weights)
        portfolio_vol = np.sqrt(np.dot(weights.T, np.dot(sigma, weights)))
        weighted_vols = np.dot(weights, asset_vols)
        diversification_ratio = weighted_vols / (portfolio_vol + 1e-8)
        return -diversification_ratio

    # Constraints: sum of weights = 1
    constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}

    # Bounds: weights between 0 and 1
    bounds = tuple((0, 1) for _ in range(num_assets))

    # Initial guess
    initial_weights = np.ones(num_assets) / num_assets

    # Optimize
    result = minimize(neg_diversification_ratio, initial_weights, bounds=bounds, constraints=constraints)
    
    # Ensure the solution is valid
    if not result.success:
        return initial_weights
    return result.x


In [18]:
def average_metrics(metrics_list):
    """
    Calculates the average of each metric in the list of metrics.
    """
    avg_metrics = {}
    keys = metrics_list[0].keys()
    for key in keys:
        avg_metrics[key] = np.mean([m[key] for m in metrics_list])
    return avg_metrics


In [19]:
# %%
# Define testing periods
training_end_dates = ['2010-12-31', '2012-12-31', '2014-12-31', '2016-12-31', '2018-12-31']
testing_start_dates = ['2011-01-01', '2013-01-01', '2015-01-01', '2017-01-01', '2019-01-01']
testing_end_dates = ['2012-12-31', '2014-12-31', '2016-12-31', '2018-12-31', '2020-04-30']

periods = list(zip(training_end_dates, testing_start_dates, testing_end_dates))

# Initialize lists to store performance metrics for each model
lstm_metrics = []
mvo_metrics = []
md_metrics = []

initial_cash = 100000

In [ ]:
for period in periods:
    training_end, testing_start, testing_end = period
    print(f"\nProcessing period: Training up to {training_end}, Testing from {testing_start} to {testing_end}")

    # Get training data
    training_data = data.loc[:training_end].copy()
    # Get testing data
    testing_data = data.loc[testing_start:testing_end].copy()

    # Ensure we have enough data
    if len(testing_data) < 50:
        print("Not enough data for testing period")
        continue

    # LSTM Model
    lstm_model = Model()
    # Train the model and get allocation
    allocation = lstm_model.get_allocations(training_data)

    # Ensure allocation sums to 1
    allocation = allocation / np.sum(allocation)

    # Get testing returns
    testing_returns = testing_data.pct_change().dropna()
    # Align the returns with allocation
    testing_returns = testing_returns.loc[:, training_data.columns]

    # Calculate portfolio returns for LSTM
    portfolio_values_lstm = [initial_cash]
    prev_weights_lstm = allocation  # Assuming the initial allocation from the model
    for ret in testing_returns.values:
        # Compute transaction cost
        transaction_cost = C * np.sum(np.abs(allocation - prev_weights_lstm))
        # Update portfolio value
        new_value = portfolio_values_lstm[-1] * (1 + np.dot(allocation, ret)) - transaction_cost * portfolio_values_lstm[-1]
        portfolio_values_lstm.append(new_value)
        # Update previous weights
        prev_weights_lstm = allocation

    # Remove the first value (initial_cash)
    portfolio_values_lstm = portfolio_values_lstm[1:]
    # Calculate performance metrics
    metrics = calculate_metrics(portfolio_values_lstm)
    lstm_metrics.append(metrics)
    
    # Mean-Variance Optimization (MV) Model
    returns_full = data.loc[:testing_end].pct_change().dropna()
    returns_testing = data.loc[testing_start:testing_end].pct_change().dropna()
    portfolio_values_mv = [initial_cash]
    prev_weights_mv = None

    # Get indices of testing dates in returns
    testing_indices = returns_full.index.get_indexer_for(returns_testing.index)

    for idx in testing_indices:
        if idx < 50:
            continue
        # Past returns for estimation
        past_returns = returns_full.iloc[idx - 50: idx]
        mu = past_returns.mean().values
        sigma = past_returns.cov().values
        # Compute MV weights
        weights_mv = get_mv_weights(mu, sigma)
        # Today's returns
        today_return = returns_full.iloc[idx].values
        # Compute portfolio return
        portfolio_return = np.dot(weights_mv, today_return)
        # Compute transaction costs
        if prev_weights_mv is None:
            transaction_cost = C * np.sum(np.abs(weights_mv))
        else:
            transaction_cost = C * np.sum(np.abs(weights_mv - prev_weights_mv))
        # Update portfolio value
        portfolio_value = portfolio_values_mv[-1] * (1 + portfolio_return) - transaction_cost * portfolio_values_mv[-1]
        portfolio_values_mv.append(portfolio_value)
        # Update previous weights
        prev_weights_mv = weights_mv

    # Remove the first value (initial_cash)
    portfolio_values_mv = portfolio_values_mv[1:]
    # Calculate performance metrics
    metrics_mv = calculate_metrics(portfolio_values_mv)
    mvo_metrics.append(metrics_mv)

    # Maximum Diversification (MD) Model
    portfolio_values_md = [initial_cash]
    prev_weights_md = None

    for idx in testing_indices:
        if idx < 50:
            continue
        # Past returns for estimation
        past_returns = returns_full.iloc[idx - 50: idx]
        sigma = past_returns.cov().values
        # Compute MD weights
        weights_md = get_md_weights(sigma)
        # Today's returns
        today_return = returns_full.iloc[idx].values
        # Compute portfolio return
        portfolio_return = np.dot(weights_md, today_return)
        # Compute transaction costs
        if prev_weights_md is None:
            transaction_cost = C * np.sum(np.abs(weights_md))
        else:
            transaction_cost = C * np.sum(np.abs(weights_md - prev_weights_md))
        # Update portfolio value
        portfolio_value = portfolio_values_md[-1] * (1 + portfolio_return) - transaction_cost * portfolio_values_md[-1]
        portfolio_values_md.append(portfolio_value)
        # Update previous weights
        prev_weights_md = weights_md

    # Remove the first value (initial_cash)
    portfolio_values_md = portfolio_values_md[1:]
    # Calculate performance metrics
    metrics_md = calculate_metrics(portfolio_values_md)
    md_metrics.append(metrics_md)


Processing period: Training up to 2010-12-31, Testing from 2011-01-01 to 2012-12-31


Epoch 1/100


c:\Users\wiztu\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0168
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -0.0191
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: -0.0514
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -0.0859
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: -0.1016
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: -0.1099
Epoch 7/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -0.1169
Epoch 8/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: -0.1166
Epoch 9/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -0.1349
Epoch 10/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: -0.1252
Epoch 11/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -0.1356
Epoch 12/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: -0.1376
Epoch 13/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -0.1397
Epoch 14/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: -0.1413
Epoch 15/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - l

c:\Users\wiztu\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: -0.1214
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: -0.1615
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: -0.1737
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: -0.1871
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.1919
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.1972
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.2040
Epoch 8/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.2036
Epoch 9/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.2041
Epoch 10/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: -0.2085
Epoch 11/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.2118
Epoch 12/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.2124
Epoch 13/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.2099
Epoch 14/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.2126
Epoch 15/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - 

c:\Users\wiztu\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: -0.0851
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.1234
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.1347
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.1470
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: -0.1670
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: -0.1564
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.1742
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.1715
Epoch 9/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.1660
Epoch 10/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.1794
Epoch 11/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.1836
Epoch 12/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.1787
Epoch 13/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.1812
Epoch 14/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.1760
Epoch 15/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - 

c:\Users\wiztu\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: -0.0490
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.0793
Epoch 3/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: -0.0976
Epoch 4/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1225
Epoch 5/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1260
Epoch 6/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1322
Epoch 7/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1389
Epoch 8/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1415
Epoch 9/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1447
Epoch 10/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: -0.1475
Epoch 11/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1503
Epoch 12/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: -0.1543
Epoch 13/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1577
Epoch 14/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: -0.1613
Epoch 15/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - 

In [21]:
# Calculate average metrics for each model
lstm_avg_metrics = average_metrics(lstm_metrics)
mvo_avg_metrics = average_metrics(mvo_metrics)
md_avg_metrics = average_metrics(md_metrics)

In [22]:
# Print the average metrics
print("\nAverage Metrics for LSTM Model:")
for key, value in lstm_avg_metrics.items():
    print(f"{key}: {value:.4f}")

print("\nAverage Metrics for MVO Strategy:")
for key, value in mvo_avg_metrics.items():
    print(f"{key}: {value:.4f}")

print("\nAverage Metrics for MD Strategy:")
for key, value in md_avg_metrics.items():
    print(f"{key}: {value:.4f}")


Average Metrics for LSTM Model:
Annualized Return: 0.1447
Annualized Std Dev: 0.0741
Sharpe Ratio: 1.8700
Sortino Ratio: 3.1029
Max Drawdown: 0.0589
% Positive Returns: 54.9419
Profit/Loss Ratio: 1.1956

Average Metrics for MVO Strategy:
Annualized Return: -0.0000
Annualized Std Dev: 0.1221
Sharpe Ratio: 0.3684
Sortino Ratio: 0.6228
Max Drawdown: 0.1587
% Positive Returns: 55.2889
Profit/Loss Ratio: 0.8767

Average Metrics for MD Strategy:
Annualized Return: 0.0882
Annualized Std Dev: 0.0582
Sharpe Ratio: 1.8102
Sortino Ratio: 2.6354
Max Drawdown: 0.0513
% Positive Returns: 57.6328
Profit/Loss Ratio: 1.0298
